In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [ ]:
# Load data into a pandas dataframe
df = pd.read_csv("~/Telehealth/Data/all_subspecialty_geocoded_230315.csv")

In [ ]:
df=df.loc[((df["timepoint"]==2021)|
            (df["timepoint"]==2022)) &
           (df["insurance_cat2"]!="Other") &
            (df["race_ethnic_cat2"]!= "Other") &
            (df["encounter_status_name"] != "Canceled")]
data = df

In [ ]:
data

In [ ]:
data["timepoint"].value_counts()

In [ ]:
data["month_quarter"].value_counts()

In [ ]:
data["month_quarter_ord"] = data["month_quarter"].apply(lambda x: 1 if x == "Jan to Mar 2021" 
                                                          else (
                                                          2 if x== "Apr to Jun 2021" else
                                                          (3 if x== "Jul to Sep 2021" else (
                                                          4 if x== "Oct to Dec 2021" else (
                                                          5 if x== "Jan to Mar 2022" else(
                                                          6 if x == "Apr to Jun 2022" else 0))))))
data["month_quarter_ord"].value_counts()

In [ ]:
data["New_Patient"].value_counts()

In [ ]:
data["visit_weekDiff_cat"] = data.apply(lambda x: 0 if x["New_Patient"] == 1
                                                          else (
                                                          1 if x["visit_weekDiff"] < 3 else
                                                          (2 if x["visit_weekDiff"]< 12 else (3))),axis=1)
data["visit_weekDiff_cat"].value_counts()

In [ ]:
data["noshow"] = data["encounter_status_name"].apply(lambda x: 1 if x == "No Show" 
                                                          else 0)
data["noshow"].value_counts()

In [ ]:
data["encounter_status_name"].value_counts()

In [ ]:
data.groupby(["race_ethnic_cat2","is_telehealth"])["encounter_status_name"].value_counts()

In [ ]:
47094/1405374

In [ ]:
data["Household Median Income"].describe()

In [ ]:
data["Median_Income_Cat"] = data["Household Median Income"].apply(lambda x: 1 if(x<50000)
                                                                                      else (2 if x<=100000
                                                                                            else (3)))
data["Median_Income_Cat"].value_counts()

In [ ]:
297279/(297279+970774+1523871)

In [ ]:
data["insurance_cat2"].value_counts()

In [ ]:
data['insurance_cat2'].mask(data['insurance_cat2'] == 'Medicare Advantage', "Medicare", inplace=True)
data["insurance_cat2"].value_counts()

In [ ]:
data["age_at_firstVisit"]=data.groupby("ir_id")["age_at_visit"].transform("first")
data[["ir_id","age_at_visit","age_at_firstVisit"]]

In [ ]:
age_cut_labels = ['18-39', '40-64','65+']
age_cut_bins = [0, 40, 65,150]
data['age_firstVisit_cat'] = pd.cut(data['age_at_firstVisit'], bins=age_cut_bins, labels=age_cut_labels)
data['age_firstVisit_cat'].value_counts(dropna=False)

In [ ]:
data["Distance"].describe()

In [ ]:
distance_cut_labels = [1,2,3,4]
distance_cut_bins = [0,3.152,6.137,12.327,350]
data['distance_cat'] = pd.cut(data['Distance'], bins=distance_cut_bins, labels=distance_cut_labels)
data['distance_cat'].value_counts(dropna=False)

In [ ]:
data["Charlson_cat"].value_counts()

In [ ]:
#combining 1-2 as one category
data["Charlson_cat"].mask(data['Charlson_cat'] == '2', "1", inplace=True)
data["Charlson_cat"].value_counts()

In [ ]:
data["Charlson_cat"] = data["Charlson_cat"].apply(lambda x: 0 if x == "0"
                                                          else (1 if x == "1"
                                                          else(2)))
data["Charlson_cat"].value_counts()

In [ ]:
#data2.to_csv("~/Data/tableone_data_230328.csv")

In [ ]:
data.columns

In [ ]:
#data.to_csv("~/Data/tableone_data_230807.csv")

In [ ]:
fam = sm.families.Binomial()

In [ ]:
ind = sm.cov_struct.Independence()

In [ ]:
black_telehealth = data.loc[(data["is_telehealth"]==True)&(df["race_ethnic_cat2"]== "Non-Hispanic Black")]
black_inperson = data.loc[(data["is_telehealth"]==False)&(df["race_ethnic_cat2"]== "Non-Hispanic Black")]
hispanic_telehealth = data.loc[(data["is_telehealth"]==True)&(df["race_ethnic_cat2"]== "Hispanic")]
hispanic_inperson = data.loc[(data["is_telehealth"]==False)&(df["race_ethnic_cat2"]== "Hispanic")]
white_telehealth = data.loc[(data["is_telehealth"]==True)&(df["race_ethnic_cat2"]== "Non-Hispanic White")]
white_inperson = data.loc[(data["is_telehealth"]==False)&(df["race_ethnic_cat2"]== "Non-Hispanic White")]

# Black telehealth

In [ ]:
mod8 = smf.gee("noshow ~ C(provider_specialty_cat,Treatment('Primary Care')) + distance_cat + C(age_firstVisit_cat) + New_Patient + C(Median_Income_Cat)+ C(Charlson_cat)+C(gender)+C(month_quarter) + C(insurance_cat2,Treatment('Commercial'))", "ir_id", black_telehealth, cov_struct=ind, family=fam)


In [ ]:
res8 = mod8.fit()

In [ ]:
print(res8.summary())

In [ ]:
params = res8.params
conf = res8.conf_int()
conf['OR'] = params
conf.columns = ["Lower CI", "Upper CI", "OR"]
np.exp(conf)

In [ ]:
res8.summary()

# Black inperson

In [ ]:
mod8 = smf.gee("noshow ~ C(provider_specialty_cat,Treatment('Primary Care')) + distance_cat + C(age_firstVisit_cat) + New_Patient + C(Median_Income_Cat)+ C(Charlson_cat)+C(gender)+C(month_quarter) + C(insurance_cat2,Treatment('Commercial'))", "ir_id", black_inperson, cov_struct=ind, family=fam)


In [ ]:
res8 = mod8.fit()

In [ ]:
print(res8.summary())

In [ ]:
params = res8.params
conf = res8.conf_int()
conf['OR'] = params
conf.columns = ["Lower CI", "Upper CI", "OR"]
np.exp(conf)

In [ ]:
res8.summary()

# Hispanic telehealth

In [ ]:
mod8 = smf.gee("noshow ~ C(provider_specialty_cat,Treatment('Primary Care')) + distance_cat + C(age_firstVisit_cat) + New_Patient + C(Median_Income_Cat)+ C(Charlson_cat)+C(gender)+C(month_quarter) + C(insurance_cat2,Treatment('Commercial'))", "ir_id", hispanic_telehealth, cov_struct=ind, family=fam)


In [ ]:
res8 = mod8.fit()

In [ ]:
print(res8.summary())

In [ ]:
params = res8.params
conf = res8.conf_int()
conf['OR'] = params
conf.columns = ["Lower CI", "Upper CI", "OR"]
np.exp(conf)

In [ ]:
res8.summary()

# Hispanic inperson

In [ ]:
mod8 = smf.gee("noshow ~ C(provider_specialty_cat,Treatment('Primary Care')) + distance_cat + C(age_firstVisit_cat) + New_Patient + C(Median_Income_Cat)+ C(Charlson_cat)+C(gender)+C(month_quarter) + C(insurance_cat2,Treatment('Commercial'))", "ir_id", hispanic_inperson, cov_struct=ind, family=fam)


In [ ]:
res8 = mod8.fit()

In [ ]:
print(res8.summary())

In [ ]:
params = res8.params
conf = res8.conf_int()
conf['OR'] = params
conf.columns = ["Lower CI", "Upper CI", "OR"]
np.exp(conf)

In [ ]:
res8.summary()

# White telelehealth

In [ ]:
mod8 = smf.gee("noshow ~ C(provider_specialty_cat,Treatment('Primary Care')) + distance_cat + C(age_firstVisit_cat) + New_Patient + C(Median_Income_Cat)+ C(Charlson_cat)+C(gender)+C(month_quarter) + C(insurance_cat2,Treatment('Commercial'))", "ir_id", white_telehealth, cov_struct=ind, family=fam)


In [ ]:
res8 = mod8.fit()

In [ ]:
print(res8.summary())

In [ ]:
params = res8.params
conf = res8.conf_int()
conf['OR'] = params
conf.columns = ["Lower CI", "Upper CI", "OR"]
np.exp(conf)

In [ ]:
res8.summary()

# White inperson

In [ ]:
mod8 = smf.gee("noshow ~ C(provider_specialty_cat,Treatment('Primary Care')) + distance_cat + C(age_firstVisit_cat) + New_Patient + C(Median_Income_Cat)+ C(Charlson_cat)+C(gender)+C(month_quarter) + C(insurance_cat2,Treatment('Commercial'))", "ir_id", white_inperson, cov_struct=ind, family=fam)


In [ ]:
res8 = mod8.fit()

In [ ]:
print(res8.summary())

In [ ]:
params = res8.params
conf = res8.conf_int()
conf['OR'] = params
conf.columns = ["Lower CI", "Upper CI", "OR"]
np.exp(conf)

In [ ]:
res8.summary()